In [ ]:
#import boto.ec2 as boto3
from openalpr import Alpr
import sys
import cv2 as cv2
import matplotlib.pyplot as plt
import time

In [ ]:
import boto3

In [ ]:
config = "/home/openalpr/src/build/config/openalpr.conf"
runtimedata_path = "/home/openalpr/runtime_data"

In [ ]:
alpr = Alpr("us", config, runtimedata_path)

In [ ]:
if not alpr.is_loaded():
    print("Error loading OpenALPR")
    sys.exit(1)

In [ ]:
alpr.set_top_n(1)

In [ ]:
def show(image):
    plt.figure(figsize=(5, 5))
    plt.imshow(image)
    plt.show()

In [ ]:
import json
class DecimalEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, decimal.Decimal):
            if o % 1 > 0:
                return float(o)
            else:
                return int(o)

In [ ]:
def InsertLiscencePlateDB(platenumber,EntryTime):
    #print(platenumber,time)
    dynamodb = boto3.resource('dynamodb')
    table = dynamodb.Table('LicensePlate')
    response = table.put_item(
       Item={
            'Licenseplate': platenumber,
            'EntryTime': EntryTime
        }
    )
    #print("PutItem succeeded:")
   # print(json.dumps(response, indent=4, cls=DecimalEncoder))

In [ ]:
def UpdateLiscencePlateEntryDB(platenumber,entrytime):
    #print(platenumber,time)
    dynamodb = boto3.resource('dynamodb')
    table = dynamodb.Table('LicensePlate')
    table.update_item(
    Key={
        'Licenseplate': platenumber
    },
    UpdateExpression='SET EntryTime = :val1',
    ExpressionAttributeValues={
        ':val1': entrytime
    }
    )

In [ ]:
def UpdateLiscencePlateExitDB(platenumber,exittime):
    #print(platenumber,time)
    #exit_time = time.strftime("%m-%d-%Y %H:%M:%S")
    dynamodb = boto3.resource('dynamodb')
    table = dynamodb.Table('LicensePlate')
    table.update_item(
    Key={
       'Licenseplate': platenumber
    },
    UpdateExpression='SET ExitTime = :val1',
    ExpressionAttributeValues={
        ':val1': exittime
    }
)

In [ ]:
#Licence Plate Detection for car entry video
VIDEO_SOURCE = "CarEntryH2.MOV"
frameskip=5
stream = cv2.VideoCapture(VIDEO_SOURCE)
count = 0
plates =[]
confidence ={}
while stream.isOpened():
    success,image= stream.read()
    if not success:
        print("video done")
        break
    if count==0:
        numFrameToSave = 30
    else:
        numFrameToSave = count*30
        
    #print('Position:', int(stream.get(cv2.CAP_PROP_POS_FRAMES)))
    stream.set(cv2.CAP_PROP_POS_FRAMES,numFrameToSave)
    #print('NewPosition:', int(stream.get(cv2.CAP_PROP_POS_FRAMES)))
    count = count + 1
    #show(image)
    results = alpr.recognize_ndarray(image)
    i = 0
    for i, plate in enumerate(results['results']):
            print('Plate #{}: {:7s} ({:.2f}%)'.format(i, plate['plate'].upper(), plate['confidence']))
            if plate['confidence']>88 and len(plate['plate'])==7:
                plate1=plate['plate'].upper()
                if (plate1 in plates): 
                    continue
                else:
                    plates.append(plate1)
                    InsertLiscencePlateDB(plate1,time.strftime("%m-%d-%Y %H:%M:%S") )
           # confidence.append(topMost['confidence'])
    #print(count)
    #if count == 20:
        #break
    count = count + 1
print(plates)
#print(unique(plates)) 

In [ ]:
VIDEO_SOURCE = "CarExitK2.MOV"
frameskip=5
stream = cv2.VideoCapture(VIDEO_SOURCE)
count = 0
plates1 =[]
confidence ={}
while stream.isOpened():
    success,image= stream.read()
    if not success:
        print("video done")
        break
    if count==0:
        numFrameToSave = 30
    else:
        numFrameToSave = count*30
        
    #print('Position:', int(stream.get(cv2.CAP_PROP_POS_FRAMES)))
    stream.set(cv2.CAP_PROP_POS_FRAMES,numFrameToSave)
    #print('NewPosition:', int(stream.get(cv2.CAP_PROP_POS_FRAMES)))
    count = count + 1
    #show(image)
    results = alpr.recognize_ndarray(image)
    i = 0
    for i, plate in enumerate(results['results']):
            print('Plate #{}: {:7s} ({:.2f}%)'.format(i, plate['plate'].upper(), plate['confidence']))
            if plate['confidence']>88 and len(plate['plate'])==7:
                plate2=plate['plate'].upper()
                if (plate2 in plates1): 
                    continue
                else:
                    plates1.append(plate2)
                    UpdateLiscencePlateExitDB(plate2,time.strftime("%m-%d-%Y %H:%M:%S"))
           # confidence.append(topMost['confidence'])
    #print(count)
    #if count == 20:
        #break
    count = count + 1
print(plates1)
#print(unique(plates)) 